UMGC DATA620 (Prof P Menta) Infographic Assignment data retrieval

Querying stackoverflow data for Python ML related tags, tags by year CSV used for Tableau and WordArt external apps.

Forked from https://www.kaggle.com/paultimothymooney/how-to-query-the-stack-overflow-data


In [1]:
import bq_helper
from bq_helper import BigQueryHelper
stackOverflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="stackoverflow")

In [2]:
query = """SELECT
  EXTRACT(YEAR FROM creation_date) AS Year,
  COUNT(*) AS Number_of_Questions,
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
WHERE
     tags LIKE "%python%" AND (REGEXP_CONTAINS(tags, r"tensorflow|pytorch|sklearn|caffe|mxnet|keras|opencv|neural"))
GROUP BY
  Year
HAVING
  Year > 2012
ORDER BY
  Year;
        """
response = stackOverflow.query_to_pandas_safe(query)
response.head(10)

,Year,Number_of_Questions
0,2013,746
1,2014,1004
2,2015,1826
3,2016,5369
4,2017,11961
5,2018,16576
6,2019,20414


In [3]:
query = """SELECT
  EXTRACT(YEAR FROM creation_date) AS creation_date,
  tags
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
WHERE
    tags LIKE "%python%" AND (REGEXP_CONTAINS(tags, r"tensorflow|pytorch|sklearn|caffe|mxnet|keras|opencv|neural"))
    AND EXTRACT(YEAR FROM creation_date) > 2012
        """
response = stackOverflow.query_to_pandas_safe(query)
response.head(10)


,creation_date,tags
0,2015,python|machine-learning|tensorflow|mnist
1,2015,python-2.7|opencv3.0
2,2015,python|opencv|matplotlib|raspberry-pi
3,2016,python|machine-learning|neural-network|keras
4,2015,python|macos|opencv|numpy|installation
5,2016,python|serialization|machine-learning|neural-n...
6,2016,python|machine-learning|tensorflow
7,2015,opencv|python-3.x
8,2015,python|neural-network|deep-learning|caffe|pycaffe
9,2015,python|opencv|contour


In [4]:
response['splits'] = response['tags'].apply(lambda x: x.split("|"))
response.head()

,creation_date,tags,splits
0,2015,python|machine-learning|tensorflow|mnist,"[python, machine-learning, tensorflow, mnist]"
1,2015,python-2.7|opencv3.0,"[python-2.7, opencv3.0]"
2,2015,python|opencv|matplotlib|raspberry-pi,"[python, opencv, matplotlib, raspberry-pi]"
3,2016,python|machine-learning|neural-network|keras,"[python, machine-learning, neural-network, keras]"
4,2015,python|macos|opencv|numpy|installation,"[python, macos, opencv, numpy, installation]"


In [5]:
import pandas as pd
new = []
for idx, row in response.iterrows():
    year = row['creation_date']
    for item in row['splits']:
        t = {}
        t['year'] = year
        t['tag']  = item
        new.append(t)
df = pd.DataFrame(new)
df.head()

,tag,year
0,python,2015
1,machine-learning,2015
2,tensorflow,2015
3,mnist,2015
4,python-2.7,2015


In [6]:
df.to_csv("tags_by_year.csv", index=False)